In [63]:
## Loading relevant packages
library(slackr)
library(AirSensor)
library(MazamaSpatialUtils)
library(lubridate)
library(dplyr)

## Loading Spatial data 
## More details on Air Sensor Git hub page if it's not setup already
initializeMazamaSpatialUtils("/home/ubuntu/Git/Sensor_Alert/Spatial_data")

## Settng the Country Code API link and look back days 
pas <- pas_createNew(countryCodes = "IN",
                     baseUrl = 'https://www.purpleair.com/json?all=true',lookbackDays = 30,
                     includePWFSL = F)
## Changing the time zone to Asia/Kolkata
pas$lastSeenDate <- with_tz(pas$lastSeenDate,tz="Asia/Kolkata")



In [64]:
Colocated_Sensors =read.csv("/home/ubuntu/Git/Sensor_Alert/Data/25_Colocated_Sensors_07Oct21.csv")
SAMOSA_pas = pas%>% pas_filter(!grepl("B",label),grepl("SAMOSA",label))
head(Colocated_Sensors)


,Sensor_name,Dongles,MacID,Status
,<fct>,<fct>,<fct>,<fct>
1,SAMOSA_0014,SAMOSA1,40:f5:20:1e:58:73,Done
2,SAMOSA_0019,SAMOSA1,40:f5:20:1d:25:c2,Done
3,SAMOSA_0033,SAMOSA2,10:52:1c:42:40:b4,Done
4,SAMOSA_0050,SAMOSA1,10:52:1c:41:66:a8,Done
5,SAMOSA_0052,SAMOSA1,10:52:1c:41:f0:cc,Done
6,SAMOSA_0055,SAMOSA2,10:52:1c:41:9f:a2,Done


In [65]:
#Send notification to slack channel
time <- Sys.time()
## Changing the time zone to UTC
time <- with_tz(time,tz="Asia/Kolkata")
## Creating 5 minutes lag for the test (If any sensor is inactive for more than an hour we send an alert)
time_check = time-minutes(10)
## Finding any inactive sensors
Inactive_sensors = SAMOSA_pas %>%mutate(Sensor_name=label)%>% filter(label %in% Colocated_Sensors$Sensor_name) %>% 
filter(time_check>lastSeenDate) %>% left_join(Colocated_Sensors,by="Sensor_name") %>%
select(Sensor_name,lastSeenDate,Dongles)
head(Inactive_sensors)
dim(Inactive_sensors)[1]

Sensor_name,lastSeenDate,Dongles
<chr>,<dttm>,<fct>
SAMOSA_0112,2021-09-16 10:41:02,SAMOSA15


[1] 1

In [66]:
## Adding the R environment file. This file contains the API used for analysis
readRenviron("/home/ubuntu/Git/API.Renviron")

slackr_setup(channel = "pa_sensor_alert",
             bot_user_oauth_token =  Sys.getenv('bot_user_oauth_token'),
             incoming_webhook_url = Sys.getenv('incoming_webhook_url'))

[1] "Successfully connected to Slack"

In [67]:
if (dim(Inactive_sensors)[1] > 0){
    
    slackr("Time of Query",time)
    slackr("Total Inactive sensors", dim(Inactive_sensors)[1])
    ## Sending the Slack alert if the sensors is inactive at any moment
    slackr("Name of Inactive sensors at the moment", pander::pandoc.table(Inactive_sensors))
}